In [1]:
#!pip install torch-geometric
#!pip install pytorch-lightning
#!pip install h5py
#!pip install matplotlib
#!pip install tensorboard

In [2]:
#%%
import sys
sys.path.append('/home/fs71922/hessl3/data/ML_Luttinger/LuttingerWard_from_ML/src/')
import torch
# from models import models as models
# from wrappers import wrapers
from torch.utils.data import DataLoader
import load_data
import datetime
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.plugins.environments import LightningEnvironment
import json
import os
import numpy as np
import h5py


def create_datasets(config):
    
    PATH = config["PATH_TRAIN"]
    f = h5py.File(PATH, 'r')
    #data = np.array(f["train"]['data'])
    #print("************************************")
    #("************************************")
    #print("************************************")
    #rint("************************************")
    #print("Size of dataset: ", data.shape)
    #print("************************************")
    #print("************************************")
    #print("************************************")
    #print("************************************")
    #train, validation = torch.utils.data.random_split(data, [int(data.__len__()*0.8), int(data.__len__())-int(data.__len__()*0.8)], generator=torch.Generator().manual_seed(42))

    if config["TRAINDATA"]==config["VALIDATIONDATA"]:
        data = np.array(f[config["TRAINDATA"]])
        train, validation = torch.utils.data.random_split(data, [int(data.__len__()*config["SPLIT"]), int(data.__len__())-int(data.__len__()*config["SPLIT"])], generator=torch.Generator().manual_seed(42))
    else:
        train = np.array(f[config["TRAINDATA"]])
        validation = np.array(f[config["VALIDATIONDATA"]])
    return train, validation


def train():
    ### JSON File contains full information about entire run (model, data, hyperparameters)
    ### TODO 
    MODEL_NAME = "AUTO_ENCODER_LW_isoiso"
    config = json.load(open('../configs/confmod_auto_encoder.json'))[MODEL_NAME]

    ''' Dataloading '''
    train_data, validation_data = create_datasets(config)
    train_data = np.array(train_data)
    validation_data = np.array(validation_data)

    ### > Single HDF5 file containing training and validation data 
    ld = __import__("load_data", fromlist=['object'])
    # data_set = load_data.Dataset_ae(config)
    train_set = getattr(ld, config["DATA_LOADER"])(config, train_data)
    validation_set = getattr(ld, config["DATA_LOADER"])(config, validation_data)

    train_dataloader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True)
    validation_dataloader = DataLoader(validation_set, batch_size=config["batch_size"], shuffle=True)


    ''' Model setup '''
    wrapers = __import__("wrappers.wrapers", fromlist=['object'])#.wrapers
    model = getattr(wrapers, config["MODEL_WRAPER"])(config)

    ''' Model loading from save file '''
    if config["continue"] == True:
        SAVEPATH = config["SAVEPATH"]
        checkpoint = torch.load(SAVEPATH)
        model.load_state_dict(checkpoint['state_dict'])
        print(" >>> Loaded checkpoint")


    ''' Logging and saving '''
    DATA_NAME = os.path.splitext(os.path.basename(config["PATH_TRAIN"]))[0]
    print(DATA_NAME)
    
    PATH = "/home/fs71922/hessl3/data/ML_Luttinger/saves/"
    CONFIGURATION = f"{PATH}/GTrend/isoiso_AD/{DATA_NAME}/save_{config['MODEL_NAME']}_BS{config['batch_size']}_{datetime.datetime.now().date()}"
    logger = TensorBoardLogger(PATH, name=CONFIGURATION)
    
    early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", min_delta=0.00, patience=20, verbose=False)
    checkpoint_callback = ModelCheckpoint(save_top_k=-1)

    
    # ### '''Define (pytorch_lightning) Trainer '''
    # ### > SLURM Training
    # trainer = pl.Trainer(max_epochs=config["epochs"], 
                        # accelerator=config["device_type"], 
                        # devices=config["devices"], 
                        # num_nodes=config["num_nodes"], 
                        # strategy='ddp', 
                        # logger=logger
                        # )
    # ### > Jupyter Notebook Training
    # trainer = pl.Trainer(max_epochs=config["epochs"], 
    #                      accelerator='gpu', 
    #                      devices=1, 
    #                      strategy='auto', 
    #                      logger=logger, 
    #                      # log_every_n_steps=1, 
    #                      plugins=[LightningEnvironment()], 
    #                      callbacks=[checkpoint_callback]
    #                     )

    # ### > Jupyter Notebook CPU Training
    trainer = pl.Trainer(max_epochs=20, accelerator='gpu', devices=1, strategy='auto', logger=logger, plugins=[LightningEnvironment()])
    
    # ''' Train '''
    trainer.fit(model, train_dataloader, validation_dataloader)
    #trainer.fit(model, train_dataloader)
    
    
    # ### ''' Saving configuration file into log folder ''' 
    LOGDIR = trainer.log_dir
    json_object = json.dumps(config, indent=4)
    with open(LOGDIR+"/config.json", "w") as outfile:
        outfile.write(json_object)




def main():
    train()

if __name__ == '__main__':
    main()
# %%


/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


U2c0_b30_N250000


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /home/fs71922/hessl3/data/ML_Luttinger/saves//GTrend/isoiso_AD/U2c0_b30_N250000/save_auto_encoder_BS10_2024-12-06


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
jac = vmap(jacrev(f), in_dims=(0,None,None))(batch[0],params,self.model)

In [ ]:
def tuple_to_dict_parameters(model: nn.Module, params: tuple[torch.nn.Parameter, ...]) -> OrderedDict[str, torch.nn.Parameter]:
    keys = list(dict(model.named_parameters()).keys())
    values = list(params)
    return OrderedDict(({k:v for k,v in zip(keys, values)}))

def f(x: torch.Tensor, params: dict[str, torch.nn.Parameter],model) -> torch.Tensor:
    return functional_call(model, params, (x, ))

batch0 = np.random.random((10, 2000)) + 1j * np.random.random((10, 2000))

#generic model with auto-differentiation
class model_wraper_generic_AD(L.LightningModule):
    ''' First part of convergence model. Wraper to train actual auto-encoding with automatic differentiation, i.e. the input is Green's function output is Luttinger Ward functional. '''
    def __init__(self, config):
        super().__init__()
        module = __import__("models.models", fromlist=['object'])
        self.model = getattr(module, config["MODEL_NAME"])(config)
        self.criterion_mse = nn.MSELoss()
        self.config = config
        self.val_pred = []
        self.val_loss = []
        self.train_loss = 0
        self.validation_loss = 0

    def forward(self, batch: torch.Tensor):
        return self.model(batch)

    def training_step(self, batch: torch.Tensor, batch_idx: int) -> torch.Tensor:
        print("training:")
        pred = self.forward(batch[0]) #this is the output of the model
        params = tuple_to_dict_parameters(self.model,tuple(self.model.parameters()))
        #if LW is real:
        #jac = vmap(jacrev(f), in_dims=(0,None,None))(batch[0],params,self.model)
        #jac = torch.squeeze(jac)
        #if real and imaginary part of G are an input:
        #Sig = torch.cat([jac[:,:int(batch[0].shape[1]/2)],-jac[:,int(batch[0].shape[1]/2):]], axis=1)/2

        #if LW is complex:
        jac = vmap(jacrev(f), in_dims=(0,None,None))(batch[0],params,self.model)
        #print('jac shape',jac.shape)
        #if phi is holomorph and it is the same as for functions all these three things should be equivalent
        #deriviative with respect to imaginary part
        #Sig = torch.cat([jac[:,1,int(batch[0].shape[1]/2):], -jac[:,0,int(batch[0].shape[1]/2):]], axis=1)
        #deriviative with respect to both real and imaginary part
        print('shape jac',jac.shape)
        Sig = torch.cat([jac[:,0,:int(batch[0].shape[1]/2)]+jac[:,1,int(batch[0].shape[1]/2):], jac[:,1,:int(batch[0].shape[1]/2)]-jac[:,0,int(batch[0].shape[1]/2):]],axis=1)/2
        #deriviative with respect to real part
        #Sig = torch.cat([jac[:,0,:int(batch[0].shape[1]/2)], jac[:,1,:int(batch[0].shape[1]/2)]], axis=1)
        target = batch[1]
        loss = self.criterion_mse(Sig, target)
        self.log('train_loss', loss.item())
        return loss